In [ ]:
import requests
import re
import tabula
import pandas as pd

# Download of Module Handbooks

In [ ]:
pdf_url_bachelor_business_management_and_economics = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-184-H-2008.pdf"
pdf_url_bachelor_information_systems = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-277-H-2021.pdf"
pdf_url_bachelor_economathematics = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-276-H-2015.pdf"
pdf_url_master_information_systems = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-j10-H-2019.pdf"
pdf_url_master_management ="https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-i45-H-2018.pdf"
pdf_url_master_international_economic_policy = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-h92-H-2018.pdf"

response = requests.get(pdf_url_bachelor_business_management_and_economics)
with open("module_handbooks/bachelor_business_management_and_economics.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded bachelor_business_management_and_economics.pdf")

response = requests.get(pdf_url_bachelor_information_systems)
with open("module_handbooks/bachelor_information_systems.pdf", "wb") as f:
    f.write(response.content)

print("Downloaded bachelor_information_systems.pdf")
response = requests.get(pdf_url_bachelor_economathematics)
with open("module_handbooks/bachelor_economathematics.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded bachelor_economathematics.pdf")

response = requests.get(pdf_url_master_information_systems)
with open("module_handbooks/master_information_systems.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_information_systems.pdf")

response = requests.get(pdf_url_master_management)
with open("module_handbooks/master_management.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_management.pdf")

response = requests.get(pdf_url_master_international_economic_policy)
with open("module_handbooks/master_international_economic_policy.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_international_economic_policy.pdf")

# Ansatz über TABULA --> Verforfen 
Reading Tables from Module Book for master_information_systems

In [ ]:
import pandas as pd
import tabula
import re

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
tables_master_is = tabula.read_pdf("module_handbooks/master_information_systems.pdf", pages='10-30', multiple_tables=True)
print(len(tables_master_is))
print(tables_master_is)

# PDF Miner Ansatz wie Felix -> über Regex

In [ ]:
import io
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFPageInterpreter, PDFResourceManager
from pdfminer.pdfpage import PDFPage


def pdf_to_text(file_path, n=0):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()

    laparams = LAParams()
    # laparams.line_margin = 2.1
    # laparams.word_margin = 0.1
    # laparams.boxes_flow = 0.01

    laparams.line_margin = 2.1
    laparams.word_margin = 0.5
    laparams.boxes_flow = 1.9


    # laparams.detect_vertical = True
    laparams.all_texts = True


    converter = TextConverter(
        resource_manager,
        fake_file_handle,
        codec="utf-8",
        laparams=laparams,
    )
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(file_path, "rb") as fh:
        for i, page in enumerate(PDFPage.get_pages(fh, caching=True, check_extractable=True)):
            if i < n:
                continue
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    converter.close()
    fake_file_handle.close()

    if text:
        return remove_header_footer(text)
    else:
        return ""


def remove_header_footer(text):
    # Split the text into lines
    lines = text.split("\n")

    # Remove lines that match the header pattern
    lines = [line for line in lines if not line.startswith("JMU Würzburg")]

    # Remove lines that match the footer pattern
    lines = [line for line in lines if not line.startswith("page")]
    lines = [line for line in lines if not re.match(r".*Master.* with 1 major.*", line)]
    lines = [line for line in lines if not re.match(r".*Master.* - 2019.*", line)]
    lines = [line for line in lines if not re.match(r".*Module Catalogue for the Subject.*", line)]
    lines = [line for line in lines if not re.match(r".*Information Systems.*", line)]
    lines = [line for line in lines if not re.match(r".*Master’s with 1 major, 120 ECTS credits.*", line)]


    # Join the remaining lines back into a single string
    clean_text = "\n".join(lines)

    return clean_text

In [ ]:
from tika import parser # pip install tika

raw = parser.from_file('module_handbooks/master_information_systems.pdf')
text= raw['content']
print(text)

In [ ]:
text = pdf_to_text("module_handbooks/master_information_systems.pdf", 9)

In [ ]:
import io
import PyPDF2

def pdf_to_text_pypdf(file_path):
    text = ""
    with open(file_path, 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)  # Use PdfReader instead of PdfFileReader
        num_pages = len(pdf_reader.pages)
        for page in range(num_pages):
            page_obj = pdf_reader.pages[page]
            text += page_obj.extract_text()
    return text

text = pdf_to_text_pypdf("module_handbooks/master_information_systems.pdf")

In [ ]:
text_no_footer_header = remove_header_footer(text)
print(text_no_footer_header)

with open("module_handbooks/master_information_systems_processed.txt", "w") as f:
    f.write(text_no_footer_header)

In [132]:
tables = re.split(r"(?=Module title)", text_no_footer_header)[1:]
print(len(tables))
print("---------------TABLE 0--------------------------")
print(tables[2])
# store to table.txt file 
with open("module_handbooks/master_information_systems_table_0.txt", "w") as f:
    f.write(tables[0])

with open("module_handbooks/master_information_systems_table_1.txt", "w") as f:
    f.write(tables[1])

136
---------------TABLE 0--------------------------
Module title Abbreviation

Project Seminar 12-PS-192-m01

Module coordinator Module offered by

Holder of the Chair of Business Management and Business 

Faculty of Business Management and Economics

ECTS Method of grading Only after succ. compl. of module(s)

15 numerical grade --

Duration Module level Other prerequisites

1 semester graduate --

Contents

Content:
In small project teams of 4 to 10 members, students will spend several months actively working on a specific and 
realistic problem with practical relevance. They will progress through several project stages including as-is analy-
sis, to-be conception and implementation of an IS solution. The project teams will be required to work indepen-
dently and will only receive advice and minor support from research assistants.
 
Reading:
will vary according to topic

Intended learning outcomes

After completing the course "Projektseminar", students will be able to
1. analyze bus

In [137]:
#old pattern
    # patterns = {
    #     "Module title": r"Module title(.*?)\sModule coordinator",
    #     "Abbreviation": r"Abbreviation(.*?)\s+Module offered by",
    #     "Method of grading": r"grading(.*?)\s\d",
    #     "ECTS": r"numerical grade*(.*?)\sModule level",
    #     "Contents": r"Contents\s+(.*?)Intended learning outcomes",
    #     "Intended learning outcomes": r"Intended learning outcomes\s+(.*?)\s+Courses",
    #     "Methods of assessment": "Method of assessment  \(type, scope, language — if other than German, examination offered — if not every semester, information on whether module is creditable for bonus\)\s+(.*?)\s+Allocation",
    #     "Workload": r"Workload\s+(.*?)\s+Teaching cycle",
    #     "Teaching cycle": r"Teaching cycle\s+(.*?)\s+Referred to in LPO I",
    #     "Referred to in LPO I": r"Referred to in LPO I  \(examination regulations for teaching-degree programmes\)+(.*?)Module appears in",
    #     "Courses": r"German\)(.*?)Method",
    #     "Module appears in": r"Module appears in\s(.*?)\sAbbreviation",
    #     "Addititional information": r"Additional information\s+(.*?)\s+Workload",
    #     "Allocation of places": r"Allocation of places\s+(.*?)\s+Additional information",
    #     "Only after succ. compl. of module(s)":r"(?<=ECTS\s)([^\n]+)\s+Other prerequisites",
    #     "Module coordinator": r"Module coordinator(.*?)\s+ECTS",
    #     # "Duration": r"Duration\s+(\d\ssemester|1\ssemester)\s",
    #     "Module offered by": r"Module offered by(.*?)Only after succ",
    # }   

import re

def extract_table_values(table_string):
    patterns = {
        "Module title": r"Abbreviation\s+(.*?)\d{2}",
        "Abbreviation": r"\b([\w\d-]+)\s+Module coordinator\b",
        "ECTS": r"module\(s\)\s+(\d*)\s",
        # add a regex pattern for 
    }

    extracted_values = {}

    # Convert the pattern.items() to lowercase
    patterns = {key.lower(): value for key, value in patterns.items()}

    # Extract values using regex
    for key, pattern in patterns.items():
        match = re.search(pattern, table_string, re.DOTALL)
        if match:
            extracted_values[key] = match.group(1).strip()

    return extracted_values

# Example usage


##Ansatz über HTML

In [149]:
import subprocess

def pdf_to_html(file_path):
    # Run the pdf2htmlEX command
    subprocess.call(["pdf2htmlEX", file_path])

    # Get the name of the HTML file
    html_file = file_path.replace(".pdf", ".html")

    # Read the contents of the HTML file
    with open(html_file, "r") as f:
        html = f.read()

    return html


ModuleNotFoundError: No module named 'pdfplumber'

In [224]:
from bs4 import BeautifulSoup

def extract_tables(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    tables = soup.find_all('table')
    return tables

with open('module_handbooks/bachelor_information_systems.html', 'r') as file:
    html_string = file.read()

# Now you have the contents of the HTML file stored in the 'html_string' variable


tables = extract_tables(html_string)
#for table in tables:
    #print(table)
    #print('---')


tables = [table for table in tables if "Abbreviation" in str(table)]
print(len(tables))
print(tables[0])



moduleTitle= tables[2].find_all('td')[2].text
abbreviation = tables[2].find_all('td')[3].text
moduleCoordinator = tables[2].find_all('td')[7].text
content = tables[2].find_all('td')[21].text
etcs = tables[1].find_all('td')[11].text
appearsIn = tables[2].find_all('tr')[-1].find_all('td')[-1].text


# get the conent of bookmark 52 in the first table
# get the content of the first cell in the first table


print(moduleTitle)
print(methodOfGrading)
print(abbreviation)
print(moduleCoordinator)
print(appearsIn)
print(content)


163
<table cellspacing="0" style="border-collapse:collapse;margin-left:5.866pt"><tr style="height:15pt"><td bgcolor="#F5F5F5" colspan="3" style="width:340pt;border-top-style:solid;border-top-width:1pt;border-left-style:solid;border-left-width:1pt;border-bottom-style:solid;border-bottom-width:1pt;border-right-style:solid;border-right-width:1pt"><p class="s12" style="padding-top: 2pt;padding-left: 3pt;text-indent: 0pt;line-height: 12pt;text-align: left;"><a name="bookmark5">Module title</a></p></td><td bgcolor="#F5F5F5" style="width:142pt;border-top-style:solid;border-top-width:1pt;border-left-style:solid;border-left-width:1pt;border-bottom-style:solid;border-bottom-width:1pt;border-right-style:solid;border-right-width:1pt"><p class="s12" style="padding-top: 2pt;padding-left: 3pt;text-indent: 0pt;line-height: 12pt;text-align: left;">Abbreviation</p></td></tr><tr style="height:24pt"><td colspan="3" style="width:340pt;border-top-style:solid;border-top-width:1pt;border-left-style:solid;bord

In [138]:
print("---------------TABLE 0--------------------------")
values = extract_table_values(tables[0])
for key, value in values.items():
    print(key + ": " + value)


print("\n---------------TABLE 1--------------------------")
values = extract_table_values(tables[1])
for key, value in values.items():
    print(key + ": " + value)

print("\n---------------TABLE 2--------------------------")
values = extract_table_values(tables[2])
for key, value in values.items():
    print(key + ": " + value)

print("\n---------------TABLE 3--------------------------")
values = extract_table_values(tables[3])
for key, value in values.items():
    print(key + ": " + value)

---------------TABLE 0--------------------------
module title: Information Processing within Organizations
abbreviation: 12-IV-161-m01
ects: 5

---------------TABLE 1--------------------------
module title: IT-Management
abbreviation: 12-M-ITM-161-m01
ects: 5

---------------TABLE 2--------------------------
module title: Project Seminar
abbreviation: 12-PS-192-m01
ects: 15

---------------TABLE 3--------------------------
module title: Information Retrieval
abbreviation: IR-161-m01
ects: 5


# Create final Dataframe with these columns

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


def process_module_handbook(tables):
    module_handbook_columns = [
        "Module title",
        "Abbreviation",
        "Module coordinator",
        "Module offered by",
        "ECTS",
        "Method of grading",
        "Duration",
        "Module Level",
        "Other Prerequisites",
        "Contents",
        "Intended Learning Outcomes",
        "Courses",
        "Method of Assessment",
        "Allocation of Places",
        "Additional Information",
        "Workload",
        "Teaching Cycle",
        "Referred to in LPO I",
        "Module appears in",
    ]
    # convert to lowercase
    module_handbook_columns = [column.lower() for column in module_handbook_columns]
    module_handbook_df = pd.DataFrame(columns=module_handbook_columns)

    for table in tables:
        values = extract_table_values(table)
        # Append the values as a new row to the module_handbook_df dataframe
        module_handbook_df = module_handbook_df.append(values, ignore_index=True)

    return module_handbook_df


df_compl = process_module_handbook(tables)
# store to csv file and excel
df_compl.to_csv("module_handbooks/master_information_systems.csv", index=False)

the preprocessing step

In [ ]:
def process_module_handbook(file_path):
    text = pdf_to_text(file_path, n=0)
    print("Done with converting to string")

    text_no_footer_header = remove_header_footer(text)
    print("Done with removing header and footer")

    tables = re.split(r"(?=Module title)", text_no_footer_header)[1:]
    print("Done with splitting into tables")

    module_handbook_columns = [
        "Module title",
        "Abbreviation",
        "Module coordinator",
        "Module offered by",
        "ECTS",
        "Method of grading",
        "Completion of Modules",
        "Duration",
        "Module Level",
        "Other Prerequisites",
        "Contents",
        "Intended Learning Outcomes",
        "Courses",
        "Method of Assessment",
        "Allocation of Places",
        "Additional Information",
        "Workload",
        "Teaching Cycle",
        "Referred to in LPO I",
        "Module appears in",
    ]
    # convert to lowercase
    module_handbook_columns = [column.lower() for column in module_handbook_columns]
    module_handbook_df = pd.DataFrame(columns=module_handbook_columns)

    for table in tables:
        values = extract_table_values(table)
        # Append the values as a new row to the module_handbook_df dataframe
        module_handbook_df = module_handbook_df.append(values, ignore_index=True)

    return module_handbook_df

df_compl = process_module_handbook("module_handbooks/master_information_systems.pdf")

In [ ]:
df_compl = process_module_handbook("module_handbooks/master_management.pdf")

In [ ]:
print(df_compl.shape)

# in column appears in, replace the \n with a comma 
df_compl["module appears in"] = df_compl["module appears in"].str.replace("\n", ", ")
# trim all strings (that means remove leading and trailing whitespaces)
df_compl = df_compl.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# what is the value of "module title=IT management" 
df_compl.to_csv("module_handbooks/master_management.csv", index=False)
print(df_compl["abbreviation"].value_counts())
